In [1]:
import pandas as pd
import numpy as num

from bokeh.models import ColumnDataSource
from bokeh.palettes import Bright6
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

output_notebook()

aisles = pd.read_csv('aisles.csv')
dep = pd.read_csv('departments.csv')
op = pd.read_csv('order_products.csv')
orde = pd.read_csv('orders.csv')
prod = pd.read_csv('products.csv')


order = orde.loc[:,['order_hour_of_day', 'user_id']].sort_values(by=['order_hour_of_day'])
plot_order = pd.pivot_table(order, values=['user_id'], index=['order_hour_of_day'], aggfunc=num.size)




Loading BokehJS ...

Ploting number of order per hours

In [2]:

source = ColumnDataSource(plot_order)

p1=figure(title = "Number of order per hours", height=600, width = 900)
p1.line(x='order_hour_of_day', y='user_id', line_width=2, color= 'black', source = source)
p1.circle(x='order_hour_of_day', y='user_id', fill_color='red', size = 8, source = source)
p1.title.text_color = "red"
p1.background_fill_color = "whitesmoke"
p1.border_fill_color = "grey"
#output_notebook()




Creating new dataframe merging products & order product

In [3]:
tri_op = op.loc[:,['order_id', 'product_id']].sort_values(by=('product_id'))
pivot_op = pd.pivot_table(tri_op, values=['order_id'], index = ['product_id'], aggfunc=num.size)
merge_op_prod = pd.merge(pivot_op, prod, how = 'inner', on=['product_id'])
merge_op_prod['order_id'] = pd.to_numeric(merge_op_prod['order_id'])
reduc_op = merge_op_prod[merge_op_prod['order_id'] > 100000]
retri_op=reduc_op.sort_values(by=('order_id'))
oop = retri_op[:6]
oop.sort_values(by=('product_id'))
df_op=oop.loc[:,['order_id', 'product_name']]

count = df_op.order_id.values.tolist()
name = df_op.product_name.values.tolist()


Ploting number of orders per product

In [4]:

source = ColumnDataSource(data=dict(name=name, count=count))

p2 = figure(x_range=name, height=600, width = 900, toolbar_location=None, title="Most order products")

p2.vbar(x='name', top='count', width=0.9, source=source, legend_field="name",
       line_color='white', fill_color=factor_cmap('name', palette=Bright6, factors=name))

p2.xgrid.grid_line_color = None
p2.legend.orientation = "horizontal"
p2.legend.location = "top_center"
p2.y_range.start = 0
p2.y_range.end = 1.600e+5




New dataframe for most frequented department

In [5]:
#reduc_op2 =
reduce = merge_op_prod[merge_op_prod['order_id'] > 55000].loc[:,['order_id', 'department_id']].sort_values(by=('department_id'))
pivot_reduce=pd.pivot_table(reduce, values=['order_id'], index = ['department_id'], aggfunc=num.size)

merge_op_dep=pd.merge(pivot_reduce, dep, how = 'inner', on=['department_id']).loc[:,['order_id', 'department']]

count_freq = merge_op_dep.order_id.values.tolist()
dep_name = merge_op_dep.department.values.tolist()


new plot for the mosts frequented departments from the 6 most ordered products

In [6]:

source = ColumnDataSource(data=dict(dep_name=dep_name, count_freq=count_freq))

p3 = figure(x_range=dep_name, height=600, width = 900, toolbar_location=None, title="Most frequented departments")

p3.vbar(x='dep_name', top='count_freq', width=0.9, source=source, legend_field="dep_name",
       line_color='white', fill_color=factor_cmap('dep_name', palette=Bright6, factors=dep_name))

p3.xgrid.grid_line_color = None
p3.legend.orientation = "horizontal"
p3.legend.location = "top_center"
p3.y_range.start = 0
p3.y_range.end = 50



New dataframe for most frequented ails in the mosts frequented departments

In [7]:
reduce_ = merge_op_prod[merge_op_prod['order_id'] > 75000].loc[:,['order_id', 'aisle_id']].sort_values(by=('aisle_id'))
pivot_reduce_=pd.pivot_table(reduce_, values=['order_id'], index = ['aisle_id'], aggfunc=num.size)
merge_op_ai=pd.merge(pivot_reduce_, aisles, how = 'inner', on=['aisle_id']).loc[:,['order_id', 'aisle']]

count_freq_ = merge_op_ai.order_id.values.tolist()
ai_name = merge_op_ai.aisle.values.tolist()


Ploting the values

In [8]:
source = ColumnDataSource(data=dict(ai_name=ai_name, count_freq_=count_freq_))

p4 = figure(x_range=ai_name, height=600, width = 900, toolbar_location='right', title="Most frequented departments")

p4.vbar(x='ai_name', top='count_freq_', width=0.9, source=source, legend_field="ai_name",
       line_color='white', fill_color=factor_cmap('ai_name', palette=Bright6, factors=ai_name))

p4.xgrid.grid_line_color = None
p4.legend.orientation = "horizontal"
p4.legend.location = "top_center"
p4.y_range.start = 0
p4.y_range.end = 50


Create a grid with all plots

In [9]:
from bokeh.layouts import gridplot

# put all the plots in a gridplot
grid_all = gridplot([[p1, p2], [p3, p4]], toolbar_location=None)
output_notebook()
show(grid_all)


Loading BokehJS ...

Convertig grid to JS

In [11]:
from bokeh.io import export_png

export_png(p1, filename="./web/plot1.png")
export_png(p2, filename="./web/plot2.png")
export_png(p3, filename="./web/plot3.png")
export_png(p4, filename="./web/plot4.png")
export_png(grid_all, filename="./web/allplots.png")



'C:\\Users\\guill\\Documents\\rendu epitech\\C-DAT-200-REM-2-1-dataviz-guillaume.roucheux\\web\\allplots.png'